In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (1000, 1000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 2000


In [5]:
from itertools import product
import numpy as np
from scipy.spatial.distance import cosine as cosine_distance
import gensim

Path to word2vec embeddings

In [6]:
word2vec_path = get_relative_path(prefix_path, DEAULT_WORDVECTORS)
os.path.exists(word2vec_path)

True

In [7]:
wordvectors = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

Load counts documents by word

In [8]:
docs_samples_word_doc_count_path = get_relative_path(data_path, DOCS_SAMPLES_WORD_DOC_COUNT)
samples_word_in_docs_count = load_pickle(docs_samples_word_doc_count_path)

Measure $idf(t, D) = log\frac{N}{\vert \left\{ d\in D:t\in d \right\} \vert}$

In [9]:
n_docs = len(docs_sample_a) + len(docs_sample_b)
t_idf = {t: np.log(n_docs/v) for t, v in samples_word_in_docs_count.items()}

In [10]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_content_for(index, samples_content, field='tokens'):
    return samples_content[index][field]

def tf_doc(terms):
    tf = Counter(terms)
    cnt = sum(tf.values())
    return {k: v/cnt for k, v in tf.items()}

m_measure_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    # Term frequencies by document
    tf_a = tf_doc(get_content_for(docs_sample_a[i], samples_content))
    tf_b = tf_doc(get_content_for(docs_sample_b[j], samples_content))
    # Term list
    t_list = list(set(tf_a) | set(tf_b))
    # TF-IDF
    tf_idf_a = np.zeros([len(t_list),])
    tf_idf_b = np.zeros([len(t_list),])
    # wordvector length 300
    centroid_a = mean = np.zeros(300, dtype=np.float64)
    centroid_b = mean = np.zeros(300, dtype=np.float64)
    for pos, t in enumerate(t_list):
        if t and t in wordvectors.vocab and t in t_idf:
            wv = wordvectors.get_vector(t)
            if t in tf_a:
                tf_idf_a[pos] = tf_a[t] * t_idf[t]
                centroid_a += wv * tf_idf_a[pos]
            if t in tf_b:
                tf_idf_b[pos] = tf_b[t] * t_idf[t]
                centroid_b += wv * tf_idf_b[pos]
            
    sum_tf_idf_a = sum(tf_idf_a)
    sum_tf_idf_b = sum(tf_idf_b)
    # Centroid
    centroid_a /= sum_tf_idf_a
    centroid_b /= sum_tf_idf_b
    
    # Measure
    measure_sim = 1.0 - cosine_distance(centroid_a, centroid_b)
    m_measure_sim[i, j] = measure_sim

print(m_measure_sim)

[[0.65111072 0.70310882 0.65259718 ... 0.72468738 0.52235828 0.54822528]
 [0.70137091 0.80561423 0.71719402 ... 0.78705137 0.57018362 0.61170953]
 [0.68522791 0.69460987 0.69879183 ... 0.73962227 0.56383926 0.70678924]
 ...
 [0.65779476 0.75228731 0.68471401 ... 0.78790058 0.57673746 0.74272504]
 [0.59932566 0.66580782 0.49443457 ... 0.67596839 0.6500183  0.55437951]
 [0.71192622 0.68136118 0.65052729 ... 0.74431948 0.55242311 0.65809202]]


In [11]:
docs_samples_measure_sim_path = get_relative_path(data_path, DOCS_SAMPLES_WORD2VEC_TFIDF)
save_pickle(m_measure_sim, docs_samples_measure_sim_path)

In [12]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_measure_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_measure_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_measure_sim)

[[0.66773104 0.72935469 0.63675897 ... 0.73157146 0.55387421 0.61209168]
 [0.71649824 0.78262252 0.68326415 ... 0.78500118 0.59432596 0.6567953 ]
 [0.69120158 0.75499128 0.65914085 ... 0.75728597 0.57334271 0.63360651]
 ...
 [0.70452175 0.76954075 0.67184318 ... 0.77187965 0.58439162 0.64581677]
 [0.59513725 0.65006135 0.56753237 ... 0.65203712 0.4936586  0.54554684]
 [0.7017605  0.76652467 0.66921001 ... 0.7688544  0.5821012  0.6432856 ]]


In [13]:
docs_samples_measure_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_WORD2VEC_TFIDF_UDV)
save_pickle(m_udv_measure_sim, docs_samples_measure_sim_udv_path)